In [22]:
import pandas as pd
import numpy as np
import os

DATANAME = 'MiddlePhalanxTW'
eps_list = [0.005*i for i in range(1,21)]
eps_name = f" ({eps_list[0]} to {eps_list[-1]})" 
dataset_name = DATANAME
result_file = os.path.join("kfold_kNN_data", "saved_results","Results on " + dataset_name + eps_name + '_new_' + '.csv')
df = pd.read_csv(result_file)

In [23]:
min_records = {'eTiOT': [], 'eTAOT' : []}
eps_records = {'eTiOT': [], 'eTAOT' : []}

def extract_best_eps(indices, alg):
    min_err = np.inf
    min_eps = 0
    for i in indices:
        if float(df[alg][i]) <= min_err and df['eps'][i] != 'Final error':
            min_err = float(df[alg][i])
            min_eps = float(df['eps'][i])
    return min_eps

eps_records['eTiOT'].append(extract_best_eps(range(len(df)), 'eTiOT'))
eps_records['eTAOT'].append(extract_best_eps(range(len(df)), 'eTAOT'))
eps_records['eTiOT'].append(extract_best_eps(range(1,len(df),2), 'eTiOT'))
eps_records['eTAOT'].append(extract_best_eps(range(1,len(df),2), 'eTAOT'))


In [24]:
eps_list = [0.01*i for i in range(1,11)]
eps_name = f" ({eps_list[0]} to {eps_list[-1]})"   
result_file_test = os.path.join("kNN_data", "saved_results","Results on " + dataset_name + eps_name + '_new_' +'.csv')
col_map = {"eTiOT": "eTiOT", "eTAOT": r" eTAOT($\omega = \omega_{\text{grid}})$"}
df2 = pd.read_csv(result_file_test)

In [25]:
df2.columns

Index(['eps', 'eTiOT', 'eTAOT$(\omega = \omega_{\text{grid}} \;/\; 5)$',
       'eTAOT$(\omega = \omega_{\text{grid}})$',
       'eTAOT$(\omega = \omega_{\text{grid}} \times 5)$'],
      dtype='object')

In [26]:
for i in range(len(df2)):
    if float(df2['eps'][i]) == eps_records['eTiOT'][1]:
        eps_records['eTiOT'].append(df2['eTiOT'][i])
    if float(df2['eps'][i]) == eps_records['eTAOT'][1]:
        eps_records['eTAOT'].append(df2[df2.columns[3]][i])

In [27]:
eps_records

{'eTiOT': [0.085, 0.09, np.float64(0.4675324675324676)],
 'eTAOT': [0.05, 0.05, np.float64(0.4545454545454546)]}

In [28]:
new_rows = []

# eps for final_error (grid=0.005)
new_rows.append({
    "eps": "eps_final_error",
    "eTiOT": eps_records["eTiOT"][0],
    "eTAOT": eps_records["eTAOT"][0]
})

# final_error for grid=0.01
new_rows.append({
    "eps": "eps_final_error_grid001",
    "eTiOT": eps_records["eTiOT"][1],
    "eTAOT": eps_records["eTAOT"][1],
})

# eps for grid=0.01
new_rows.append({
    "eps": "final_error_grid001",
    "eTiOT": eps_records["eTiOT"][2],
    "eTAOT": eps_records["eTAOT"][2],
})

# append new rows after the last row
df = pd.concat([df, pd.DataFrame(new_rows)], ignore_index=True)

# save back
df.to_csv(result_file, index=False)
print("Updated result_file with final_error + eps values.")

Updated result_file with final_error + eps values.


In [2]:
print(len(X1[0]))

176


## Concatenate TiOT into overall results

In [5]:
import pandas as pd
import numpy as np
import os
eps_list = [0.01*i for i in range(1,11)]
eps_name = f" ({eps_list[0]} to {eps_list[-1]})" 
dataset_name = "ProximalPhalanxTW"
result_file = os.path.join("kNN_data", "saved_results","Results on " + dataset_name + eps_name + '_new_' +'.csv')
TiOT_file = os.path.join("kNN_data", "saved_results","Results on " + dataset_name + eps_name + '_freq5_TiOTonly' + '.csv')
final_file = os.path.join("kNN_data", "saved_results","Results on " + dataset_name + eps_name + '_freq5_' + '.csv')
# Read both CSV files
allresult = pd.read_csv(result_file)
TiOTresult = pd.read_csv(TiOT_file)

# Replace TiOT column in file1 with TiOT column from file2
allresult["eTiOT"] = TiOTresult["eTiOT"]

# Save the modified file1
allresult.to_csv(final_file, index=False)

## End of concatenation

In [2]:
import numpy as np


def gaussian_mixture_timeseries(length, n_components=3, weights=None, means=None, stds=None, random_state=None):
    """
    Generate a Gaussian mixture time series.
    
    Parameters:
        length (int): Length of the time series.
        n_components (int): Number of Gaussian components.
        weights (list or None): Mixing weights (must sum to 1). If None, uniform weights are used.
        means (list or None): Means of Gaussians. If None, random values are used.
        stds (list or None): Standard deviations of Gaussians. If None, random values are used.
        random_state (int or None): Random seed for reproducibility.

    Returns:
        ts (ndarray): Generated time series of shape (length,).
        component_ids (ndarray): The component index chosen at each time step.
    """
    rng = np.random.default_rng(random_state)

    # Default weights
    if weights is None:
        weights = np.ones(n_components) / n_components
    weights = np.array(weights) / np.sum(weights)  # normalize
    
    # Default means and stds
    if means is None:
        means = rng.uniform(-5, 5, size=n_components)
    if stds is None:
        stds = rng.uniform(0.5, 2.0, size=n_components)

    # Sample component for each time step
    component_ids = rng.choice(n_components, size=length, p=weights)
    
    # Draw samples from corresponding Gaussians
    ts = rng.normal(means[component_ids], stds[component_ids])
    
    return ts, component_ids


# Example usage
ts, comps = gaussian_mixture_timeseries(500, n_components=2, random_state=42)
ts2, comps = gaussian_mixture_timeseries(500, n_components=2, random_state=41)




In [ ]:
import numpy as np
import ot
import pandas as pd
from collections import defaultdict
import seaborn as sns
import matplotlib.pyplot as plt
import TiOT_lib
import os

d0, pi, w, t = TiOT_lib.eTiOT(ts, ts2, eps = 0.1, freq = 20, solver='newton', verbose=False, timing=True)
d0, t

(0.15117875678506057, 0.1705291000000102)

In [4]:
d1, pi, w, t = TiOT_lib.eTiOT(ts, ts2, eps = 0.1, freq = 20,  verbose=2, timing=True, eta=1, init_stepsize=False)
d1, t

Total subiteration needed for PGD: 21 with df = -3.3838906544349e-08
Total subiteration needed for PGD: 12 with df = -9.692690699836031e-09
Total subiteration needed for PGD: 7 with df = -1.6730240306233528e-08
Total subiteration needed for PGD: 6 with df = -2.265485321028041e-09
Total subiteration needed for PGD: 5 with df = -1.988943219269259e-09
Total subiteration needed for PGD: 4 with df = -5.114201713201229e-09
TiOT-BCD Algorithm converges after 102 iterations 


(0.151178756267585, 0.30783180000000243)

In [10]:
d1, pi, t = TiOT_lib.eTAOT(ts, ts2, eps = 0.1, freq = 20,  verbose=2, timing=True)
d1, t

TAOT-BCD Algorithm converges after 22 iterations


(0.13795703386781924, 0.010080599999994888)

In [29]:
import numpy as np
import ot
import pandas as pd
from collections import defaultdict
import seaborn as sns
import matplotlib.pyplot as plt
import TiOT_lib
import os

def process_data(dataset_name, start1, start2, numpoint ):
    filepath = os.path.join("time_series_kNN", dataset_name, dataset_name + "_TRAIN.txt" )

    with open(filepath, "r") as file:
        data = [line.strip().split() for line in file]

    # Convert to numerical values if needed
    data = [[float(value) for value in row] for row in data]

    X = [row[1:] for row in data]
    X1 = X[start1: start1 + numpoint]
    X2 = X[start2: start2 + numpoint]
    return X1, X2
dataset_name = 'DistalPhalanxOutlineCorrect' 
X1, X2 = process_data(dataset_name, start1=0, start2=10, numpoint=100)


In [52]:
X, Y = X1[1], X1[18]

In [53]:
d0, pi, w, t = TiOT_lib.eTiOT(X, Y, eps = 0.05, freq = 20, solver='newton', verbose=2, timing=True)
d0, t

Newton Algorithm converges after 4 iterations with w = 0.6464018446049217, df = 8.236125106572678e-12, df2 = 0.21216957592183988 and possible stepsize 1/df2 = 4.7132110985054005
Newton Algorithm converges after 4 iterations with w = 0.6632783113863736, df = 2.933125313811513e-12, df2 = 0.38914196055325856 and possible stepsize 1/df2 = 2.5697562878551063
Newton Algorithm converges after 3 iterations with w = 0.6648066386483149, df = 1.0886902273785082e-10, df2 = 0.4320178587070564 and possible stepsize 1/df2 = 2.314719125252834
Newton Algorithm converges after 3 iterations with w = 0.6629464834398444, df = 3.5696804279866656e-10, df2 = 0.4546339588275269 and possible stepsize 1/df2 = 2.199571722664402
Newton Algorithm converges after 3 iterations with w = 0.6610864919502826, df = 4.039482246961934e-10, df2 = 0.46452281268284756 and possible stepsize 1/df2 = 2.152746803164539
TiOT-BCD Algorithm converges after 82 iterations 


(0.05539122417710113, 0.0019677000000228873)

In [28]:
d1, pi, w, t = TiOT_lib.eTiOT(X, Y, eps = 0.05, eta=0.1, freq = 20, verbose=True, timing=True, submax_iter=200)
d1, t

Total subiteration needed for PGD: 199 with df = -0.0007157010214145224
Total subiteration needed for PGD: 94 with df = -9.364147168420232e-07
Total subiteration needed for PGD: 143 with df = -9.447685307270234e-07
Total subiteration needed for PGD: 156 with df = -9.299602799469134e-07
TiOT-BCD Algorithm converges after 62 iterations 


(0.05548163755867227, 0.025340199999845936)

In [4]:
d2, pi, w, t = TiOT_lib.eTiOT(X, Y, eps = 0.1, freq = 20, verbose=True, timing=True, submax_iter=20)
d2, t

Total subiteration needed for PGD: 19 with df = -0.0480113146621186
Total subiteration needed for PGD: 19 with df = -0.04885730901578723
Total subiteration needed for PGD: 19 with df = -0.04720987836665548
Total subiteration needed for PGD: 19 with df = -0.04519027360131247
Total subiteration needed for PGD: 19 with df = -0.043055287024125755
Total subiteration needed for PGD: 19 with df = -0.04093485593669537
TiOT-BCD Algorithm converges after 101 iterations
||g-g_old|| = 0.0062967749262412545


(0.06279226372256227, 0.05121640000000127)

In [5]:
d3, pi, w, t = TiOT_lib.eTiOT(X, Y, eps = 0.05, freq = 4, solver='PGD', verbose=True, timing=True)
d3,t

Total subiteration needed for PGD: 49 with df = -0.028703110824373747
Total subiteration needed for PGD: 49 with df = -0.028788357247891097
Total subiteration needed for PGD: 49 with df = -0.028034624328462727
Total subiteration needed for PGD: 49 with df = -0.02712501912648551
Total subiteration needed for PGD: 49 with df = -0.026154621599509317
Total subiteration needed for PGD: 49 with df = -0.02516016023047627
Total subiteration needed for PGD: 49 with df = -0.0241575637782391
Total subiteration needed for PGD: 49 with df = -0.023153598924250215
Total subiteration needed for PGD: 49 with df = -0.02215067164831322
Total subiteration needed for PGD: 49 with df = -0.02114907998204344
Total subiteration needed for PGD: 49 with df = -0.020148124279724233
Total subiteration needed for PGD: 49 with df = -0.01914667124920419
Total subiteration needed for PGD: 49 with df = -0.01814345736337799
Total subiteration needed for PGD: 49 with df = -0.017137277407998065
Total subiteration needed fo

(0.03672474723124107, 0.5282674000000043)

In [30]:
np.abs(d1 -d0), np.abs(d2-d0), np.abs(d3-d0)

(8.512607840849284e-10, 7.136090306093801e-06, 2.9745295493516233e-06)

In [34]:
def f(x):
    return x*10, x*100
f(10)[-1]

1000

In [35]:
f(20)[-1]

2000